In [1]:
import cv2
import numpy as np
import random

In [2]:
def make_noise(image, prob):
    r, g, b = cv2.split(image)
    def make_noise_channel(channel):
        channel_prob = prob / image.shape[2]
        result_channel = channel.copy()
        for i in range(channel.shape[0]):
            for j in range(channel.shape[1]):
                rand = np.random.randint(0, 101)
                if rand <= channel_prob:
                    result_channel[i, j] = np.random.randint(0, 256)
        
        return result_channel
    
    result_r = make_noise_channel(r)
    result_g = make_noise_channel(g)
    result_b = make_noise_channel(b)
    
    result = cv2.merge((result_r, result_g, result_b))
    
    return result

def convol_image(image, mask):
    offset_x = int(mask.shape[0] / 2)
    offset_y = int(mask.shape[1] / 2)
    def conv_elem(i, j):
        result = 0
        for k in range(mask.shape[0]):
            for l in range(mask.shape[1]):
                result += image[i - offset_x + k, j - offset_y + l] * mask[k, l]
        return result
    result = np.zeros(image.shape)
    for i in range(offset_x, result.shape[0] - offset_x):
        for j in range(offset_y, result.shape[1] - offset_y):
            result[i, j] = conv_elem(i, j)
    return result

def corr(A, B, k, l):
    rows = A.shape[0]
    cols = A.shape[1]
    result = None
    if k < 0 and l < 0:
        return np.sum(np.sum(A[0:rows - abs(k), 0:cols - abs(l)] * B[abs(k):rows, abs(l):cols])) / (
                (rows - 1) * (cols - 1))

    elif k < 0 and l >= 0:
        return np.sum(np.sum(A[0:rows - abs(k), l:cols] * B[abs(k):rows, 0:cols - l])) / ((rows - 1) * (cols - 1))

    elif k >= 0 and l < 0:
        return np.sum(np.sum(A[k:rows, 0:cols - abs(l)] * B[0:rows - k, abs(l):cols])) / ((rows - 1) * (cols - 1))

    return np.sum(np.sum(A[k:rows, l:cols] * B[0:rows - k, 0:cols - l])) / ((rows - 1) * (cols - 1))

def get_b(image, image_noise, D):
    b = np.zeros(len(D) ** 2)
    i = 0
    
    for k in D:
        for l in D:
            b[i] = corr(image, image_noise, -k, -l)
            i += 1
    return b

def get_A(image, D):
    size = len(D) ** 2
    result = np.zeros((size, size))
    
    i = 0
    j = 0
    
    for k in D:
        for l in D:
            for n in D:
                for m in D:
                    result[i, j] = corr(image, image, k - n, l - m)
                    j += 1
            i += 1
            j = 0
    return result

def get_x(A, b):
    return np.linalg.solve(A, b).reshape(int(np.sqrt(b.shape)), int(np.sqrt(b.shape)))

def filter_image(image, image_noise, D, suffix):
    r, g, b = cv2.split(image)

    file = suffix
    
    image_noise_r, image_noise_g, image_noise_b = cv2.split(image_noise)
    
    cv2.imwrite(path + file + '_noise.jpg', image_noise)
    cv2.imwrite(path + file + '_r.jpg', r)
    cv2.imwrite(path + file + '_g.jpg', g)
    cv2.imwrite(path + file + '_b.jpg', b)
    cv2.imwrite(path + file + '_r_noise.jpg', image_noise_r)
    cv2.imwrite(path + file + '_g_noise.jpg', image_noise_g)
    cv2.imwrite(path + file + '_b_noise.jpg', image_noise_b)

    A_r = get_A(r, D)
    A_g = get_A(g, D)
    A_b = get_A(b, D)

    b_r = get_b(r, r, D)
    b_g = get_b(g, g, D)
    b_b = get_b(b, b, D)

    x_r = get_x(A_r, b_r)
    x_g = get_x(A_g, b_g)
    x_b = get_x(A_b, b_b)

    result_r = convol_image(r, x_r)
    result_g = convol_image(g, x_g)
    result_b = convol_image(b, x_b)

    cv2.imwrite(path + file + '_filter_r.jpg', result_r)
    cv2.imwrite(path + file + '_filter_g.jpg', result_g)
    cv2.imwrite(path + file + '_filter_b.jpg', result_b)

    result = cv2.merge((result_r, result_g, result_b))

    cv2.imwrite(path + file + '_filter.jpg', result)
    
    return result

def get_error(image, filtered_image):
    difference = (image - filtered_image) ** 2
    
    return (np.sum(difference) / (image.shape[0] * image.shape[1])) ** 0.5

In [3]:
path = '/home/sergei/'
D = [-3, -2, -1]

# Протестировать фильтрацию на изображении из пункта 1, осуществив КИХ-фильтрацию с использованием маски, вычисленной на основе методики, описанной в пунктах 3-5.

In [4]:
obama_no_noise_image = cv2.imread(path + 'obama.jpg')

obama_no_noise_filtered_image = filter_image(obama_no_noise_image, obama_no_noise_image, D, 'obama_no_noise')

# Повторить пункт 6 для того же изображения, предварительно зашумив его с вероятностью искажения p.

In [5]:
obama_noise_image = cv2.imread(path + 'obama.jpg')
obama_noise_image_noise = make_noise(obama_noise_image, 9)

obama_noise_filtered_image = filter_image(obama_noise_image, obama_noise_image_noise, D, 'obama_noise')


# Осуществить зашумление тестовых изображений из пункта 2 с вероятностью искажения пикселя . Осуществить КИХ фильтрацию изображений

In [6]:
lukashenko_image = cv2.imread(path + 'lukashenko.jpg')
lukashenko_image_noise = make_noise(lukashenko_image, 9)

lukashenko_filtered_image = filter_image(lukashenko_image, lukashenko_image_noise, D, 'lukashenko')

In [7]:
bicycle_image = cv2.imread(path + 'bicycle.jpg')
bicycle_image_noise = make_noise(bicycle_image, 9)

bicycle_filtered_image = filter_image(bicycle_image, bicycle_image_noise, D, 'bicycle')

# Вычислить ошибку восстановления сигнала для всех трёх изображений

In [8]:
print('Error for obama no noise {}'.format(get_error(obama_no_noise_image, obama_no_noise_filtered_image)))

Error for obama no noise 17.61224872017722


In [9]:
print('Error for obama noise {}'.format(get_error(obama_noise_image, obama_noise_filtered_image)))

Error for obama noise 17.61224872017722


In [10]:
print('Error for lukashenko {}'.format(get_error(lukashenko_image, lukashenko_filtered_image)))

Error for lukashenko 15.57689049438446


In [11]:
print('Error for bicycle {}'.format(get_error(bicycle_image, bicycle_filtered_image)))

Error for bicycle 44.8774546518174
